In [10]:
import pandas as pd

def clean_data(path):
    df = pd.read_csv(path)
    # ... тут вся очистка ...
    return df

In [11]:
df = pd.read_csv('vk_cleaned.csv')

In [12]:
# 2. Установим библиотеку для работы с SQLite
!pip install sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\Bulat\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [13]:
# Создать базу и загрузить туда данные
from sqlalchemy import create_engine

# создаём подключение к базе SQLite (файл создастся автоматически)
engine = create_engine('sqlite:///vk_cleaned.db', echo=False)

# записываем DataFrame df в таблицу vk_posts (если уже есть — перезапишем)
df.to_sql('vk_posts', con=engine, if_exists='replace', index=False)

print("Данные успешно загружены в vk_cleaned.db")

Данные успешно загружены в vk_cleaned.db


In [14]:
# пример SQL-запроса — топ-5 постов по лайкам
query = "SELECT text_len, likes, views FROM vk_posts ORDER BY likes DESC LIMIT 5"
result = pd.read_sql(query, engine)
print(result)

   text_len  likes  views
0       385   1847      0
1       149   1531      0
2         7   1113   9217
3       255    981  73404
4       221    940  71609


In [15]:
import sqlite3

# Создаём подключение к файлу базы данных (если файла нет — создаст)
conn = sqlite3.connect("vk_cleaned.db")
cursor = conn.cursor()

In [16]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS posts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    text TEXT,
    text_len INTEGER,
    comments INTEGER,
    likes INTEGER,
    reposts INTEGER,
    views INTEGER,
    is_pinned TEXT,
    attachments TEXT,
    post_source TEXT
)
''')
conn.commit()


In [17]:
# Используем Pandas, чтобы автоматизировать вставку:

import pandas as pd

# Загружаем уже очищенный CSV
df = pd.read_csv("vk_cleaned.csv")

# Записываем в базу SQLite
df.to_sql("posts", conn, if_exists="replace", index=False)
print("Данные успешно записаны в базу!")


Данные успешно записаны в базу!


In [18]:
# Проверяем, что всё записалось
cursor.execute("SELECT COUNT(*) FROM posts")
print("Всего записей в базе:", cursor.fetchone()[0])

cursor.execute("SELECT date, likes, views FROM posts LIMIT 5")
for row in cursor.fetchall():
    print(row)

Всего записей в базе: 3966
('2021-11-01 10:32:31', 417, 63145)
('2022-01-02 10:18:21', 26, 17319)
('2021-12-31 10:00:22', 87, 21783)
('2021-12-29 07:00:40', 103, 24419)
('2021-12-28 14:16:42', 28, 19849)


In [19]:
# Закроем соединение после работы
conn.close()